## Label Priming
To produce a synthetic mocap dataset for training SOMA, the algorithm requires instructions to place virtual markers on the synthetic body; i.e. markerlayout.
In the SOMA paper we use one labeled frame per *significant variation of the markerlayout* to estimate the markerlayout(s).
This is an easy requirement for a simple and well-documented dataset.
However, if there are many variations across the dataset with many markers used for the captures, as in the CMUII and Mixamo datasets of the paper, this process can get cumbersome.
The issue is that it needs a human expert in the loop to identify points in the MPC and assign them to a marker.
This would be bottleneck for major datasets.

As a remedy, we provide a "general" SOMA model trained on a super markerset, i.e. SuperSet, containing 89 common markers used in mocap setups.

<img alt="alt text" height="256" src="https://download.is.tue.mpg.de/soma/tutorials/superset.png"
title="a mocap superset with 89 markers" width="256"/>

One can blindly run the SuperSet SOMA model on given marker-based mocap point cloud data and
SOMA will assign disjoint label from the SuperSet to each point.
These labels are usually good and only one or two swaps/mislabels can happen in difficult scenarios.
Nevertheless, even the somehow perfect results, make the process of finding significant variations of the markerlayouts and labeling one frame per each much easier.
In addition, it removes the requirement of an expert mocap technician in the loop, as one gets an idea of what a point should be labeled as.

You can open [this sample c3d file](https://download.is.tue.mpg.de/soma/tutorials/SuperSet_89_markers.c3d) in
[Mokka](https://biomechanical-toolkit.github.io/mokka/)
for interactive visualization of the markers in the SuperSet.

We assume you have already gone through the  [Running SOMA on SOMA dataset](src/tutorials/runing_soma_on_soma_dataset.ipynb) tutorial and
that you have the required folder structure in place.

Let's apply our SuperSet SOMA model to the SOMA dataset's MPC data and see the results.
We assume you have already gone through the  [Running SOMA on SOMA dataset](src/tutorials/runing_soma_on_soma_dataset.ipynb) tutorial and
that you have the required folder structure in place.

Download the
[pre-trained SuperSet SOMA model](https://download.is.tue.mpg.de/download.php?domain=soma&sfile=training_experiments/V48_02_SuperSet.tar.bz2)
and place it under
```` training_experiments/V48_02_SuperSet  ````- Also download the 
[corresponding data file](https://download.is.tue.mpg.de/download.php?domain=soma&sfile=smplx/data/V48_01_SuperSet.tar.bz2) and place it at 
```` data/V48_02_SuperSet  ````-

Now let's run the SuperSet model on *SOMA_unlabeled_mpc* data that we
[obtained before](src/tutorials/runing_soma_on_soma_dataset.ipynb).
But first, let's make sure our mocaps are in good shape.

One has to make sure of two things when running SOMA on mocap point clouds.
First, the unit of the points should be known, and one should pass it to the algorithm along with the mocaps.
For c3d files, the default unit is *mm*, yet this can be different.

Furthermore, SOMA expects to receive mocaps with Z-axis up.
If that is not the case then you need to properly rotate the mocaps via
```` 'mocap.rotate': [X, Y,  Z] ```` while the xyz values are in degrees around that axis.

One way to avoid issues is to visualize your mocaps. For that, you can use
```` from moshpp.tools.mocap_interface import MocapSession  ````.

As a hint, when displaying your mocaps with MoCapSession you should always see the points from above, 
like this subject standing in T-Pose:

![alt text](https://download.is.tue.mpg.de/soma/tutorials/correct_mocap.png).

If you don't see any points then you probably need to change the units, and if the view is not from above then
play with rotation until you get it similar to the picture above.

With this quick pre-check, let's get to actual label priming.

In [7]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import os.path as osp
import numpy as np
from soma.tools.run_soma_multiple import run_soma_on_multiple_settings

In [5]:
soma_expr_id = 'V48_02_SuperSet'

soma_work_base_dir = '/home/nghorbani/Desktop/SOMA_Test'
support_base_dir = osp.join(soma_work_base_dir, 'support_files')

mocap_base_dir = osp.join(support_base_dir, 'evaluation_mocaps/original')

In [9]:
run_soma_on_multiple_settings(
    soma_expr_ids=[
        soma_expr_id,
    ],
    soma_mocap_target_ds_names=[
        'SOMA_unlabeled_mpc',
    ],
    soma_data_ids=[
        'OC_05_G_03_real_000_synt_100',
    ],
    soma_cfg={
        'soma.batch_size': 256,
        'dirs.support_base_dir':support_base_dir,
        'mocap.unit': 'mm',
        'save_c3d': True,
        'keep_nan_points': True,
        'remove_zero_trajectories': True
    },
    parallel_cfg={
        'max_num_jobs': 10,# comment to run on whole dataset
        'randomly_run_jobs': True,
    },
    run_tasks=[
        'soma',
    ],
    mocap_base_dir = mocap_base_dir,
    soma_work_base_dir=soma_work_base_dir,
    mocap_ext='.c3d'
)

run_soma_multiple:run_soma_on_multiple_settings:245 -- V48_02_SuperSet -- OC_05_G_03_real_000_synt_100 -- SOMA_unlabeled_mpc -- soma_subject1 -- throw_003 -- Submitting SOMA jobs.
parallel_tools:run_parallel_jobs:54 -- V48_02_SuperSet -- OC_05_G_03_real_000_synt_100 -- SOMA_unlabeled_mpc -- soma_subject1 -- throw_003 -- #Job(s) submitted: 69
parallel_tools:run_parallel_jobs:58 -- V48_02_SuperSet -- OC_05_G_03_real_000_synt_100 -- SOMA_unlabeled_mpc -- soma_subject1 -- throw_003 -- max_num_jobs is set to 10. choosing the first #Job(s): 10
parallel_tools:run_parallel_jobs:67 -- V48_02_SuperSet -- OC_05_G_03_real_000_synt_100 -- SOMA_unlabeled_mpc -- soma_subject1 -- throw_003 -- Will run the jobs in random order.
soma_processor:__init__:98 -- V48_02_SuperSet -- OC_05_G_03_real_000_synt_100 -- SOMA_unlabeled_mpc -- soma_subject1 -- run_002 -- Starting SOMA on /home/nghorbani/Desktop/SOMA_Test/support_files/evaluation_mocaps/original/SOMA_unlabeled_mpc/soma_subject1/run_002.c3d
model_loade

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  4.08it/s]

soma_processor:apply_soma:234 -- V48_02_SuperSet -- OC_05_G_03_real_000_synt_100 -- SOMA_unlabeled_mpc -- soma_subject1 -- run_002 -- SOMA was performing at inf Hz.


soma_processor:label_tracklets:388 -- V48_02_SuperSet -- OC_05_G_03_real_000_synt_100 -- SOMA_unlabeled_mpc -- soma_subject1 -- run_002 -- 53 labels detected
soma_processor:label_tracklets:395 -- V48_02_SuperSet -- OC_05_G_03_real_000_synt_100 -- SOMA_unlabeled_mpc -- soma_subject1 -- run_002 -- Tracklet labeling yielded 99.07% non-zero mocap.
soma_processor:run_soma_once:446 -- V48_02_SuperSet -- OC_05_G_03_real_000_synt_100 -- SOMA_unlabeled_mpc -- soma_subject1 -- run_002 -- Created /home/nghorbani/Desktop/SOMA_Test/training_experiments/V48_02_SuperSet/OC_05_G_03_real_000_synt_100/evaluations/soma_labeled_mocap_tracklet/SOMA_unlabeled_mpc/soma_subject1/run_002.pkl
soma_processor:run_soma_once:455 -- V48_02_SuperSet -- OC_05_G_03_real_000_synt_100 -- SOMA_unlabeled_mpc -- soma_subject1 -- run_002 -- Created /home/nghorbani/Desktop/SOMA_Test/training_experiments/V48_02_SuperSet/OC_05_G_03_real_000_synt_100/evaluations/soma_labeled_mocap_tracklet/SOMA_unlabeled_mpc/soma_subject1/run_00

/is/ps3/nghorbani/opt/anaconda3/envs/soma-public/lib/python3.7/site-packages/moshpp-3.0-py3.7.egg/moshpp/tools/c3d.py:902: UserWarning: missing parameter ANALOG:LABELS
  warnings.warn('missing parameter {}'.format(name))
/is/ps3/nghorbani/opt/anaconda3/envs/soma-public/lib/python3.7/site-packages/moshpp-3.0-py3.7.egg/moshpp/tools/c3d.py:902: UserWarning: missing parameter ANALOG:DESCRIPTIONS
  warnings.warn('missing parameter {}'.format(name))


...

soma_processor:apply_soma:234 -- V48_02_SuperSet -- OC_05_G_03_real_000_synt_100 -- SOMA_unlabeled_mpc -- soma_subject1 -- kick_002 -- SOMA was performing at 2457.0 Hz.


soma_processor:label_tracklets:388 -- V48_02_SuperSet -- OC_05_G_03_real_000_synt_100 -- SOMA_unlabeled_mpc -- soma_subject1 -- kick_002 -- 53 labels detected
soma_processor:label_tracklets:395 -- V48_02_SuperSet -- OC_05_G_03_real_000_synt_100 -- SOMA_unlabeled_mpc -- soma_subject1 -- kick_002 -- Tracklet labeling yielded 99.02% non-zero mocap.
soma_processor:run_soma_once:446 -- V48_02_SuperSet -- OC_05_G_03_real_000_synt_100 -- SOMA_unlabeled_mpc -- soma_subject1 -- kick_002 -- Created /home/nghorbani/Desktop/SOMA_Test/training_experiments/V48_02_SuperSet/OC_05_G_03_real_000_synt_100/evaluations/soma_labeled_mocap_tracklet/SOMA_unlabeled_mpc/soma_subject1/kick_002.pkl
soma_processor:run_soma_once:455 -- V48_02_SuperSet -- OC_05_G_03_real_000_synt_100 -- SOMA_unlabeled_mpc -- soma_subject1 -- kick_002 -- Created /home/nghorbani/Desktop/SOMA_Test/training_experiments/V48_02_SuperSet/OC_05_G_03_real_000_synt_100/evaluations/soma_labeled_mocap_tracklet/SOMA_unlabeled_mpc/soma_subject1/k

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  3.85it/s]

soma_processor:apply_soma:234 -- V48_02_SuperSet -- OC_05_G_03_real_000_synt_100 -- SOMA_unlabeled_mpc -- soma_subject1 -- throw_003 -- SOMA was performing at inf Hz.


soma_processor:label_tracklets:388 -- V48_02_SuperSet -- OC_05_G_03_real_000_synt_100 -- SOMA_unlabeled_mpc -- soma_subject1 -- throw_003 -- 53 labels detected
soma_processor:label_tracklets:395 -- V48_02_SuperSet -- OC_05_G_03_real_000_synt_100 -- SOMA_unlabeled_mpc -- soma_subject1 -- throw_003 -- Tracklet labeling yielded 99.89% non-zero mocap.
soma_processor:run_soma_once:446 -- V48_02_SuperSet -- OC_05_G_03_real_000_synt_100 -- SOMA_unlabeled_mpc -- soma_subject1 -- throw_003 -- Created /home/nghorbani/Desktop/SOMA_Test/training_experiments/V48_02_SuperSet/OC_05_G_03_real_000_synt_100/evaluations/soma_labeled_mocap_tracklet/SOMA_unlabeled_mpc/soma_subject1/throw_003.pkl
soma_processor:run_soma_once:455 -- V48_02_SuperSet -- OC_05_G_03_real_000_synt_100 -- SOMA_unlabeled_mpc -- soma_subject1 -- throw_003 -- Created /home/nghorbani/Desktop/SOMA_Test/training_experiments/V48_02_SuperSet/OC_05_G_03_real_000_synt_100/evaluations/soma_labeled_mocap_tracklet/SOMA_unlabeled_mpc/soma_subje

In the output log above, take a note of the green line with the content like below:

```` soma_processor:label_tracklets:395 -- V48_02_SuperSet -- OC_05_G_03_real_000_synt_100 -- SOMA_unlabeled_mpc -- soma_subject1 -- dance_003 -- Tracklet labeling yielded 99.67% non-zero mocap. ````

One way to look for successful labeling is the amount of non-zero markers the process yielded; i.e. *Tracklet labeling yielded 99.67% non-zero mocap.*
This is just sign that things *might have been gone correct.*
Now if this value was down to ~50% then most probably you have turned off the 'remove_zero_trajectories'.
Set it to True and the number will go up; i.e. ```` 'remove_zero_trajectories': True ````.
Alternatively, the number can be very low, i.e. <20%, in which case most probably you have set the units or rotation wrong.

Go ahead and check for the c3d files located at:
```` training_experiments/V48_02_SuperSet/OC_05_G_03_real_000_synt_100/evaluations/soma_labeled_mocap_tracklet/SOMA_unlabeled_mpc ````.
They should already be in good shape.
Choose one, and open it in [Mokka](https://biomechanical-toolkit.github.io/mokka/).
Have in mind that SuperSet model is blind to the actual markerlayout used for the capture.

That is why we did not train a new model for our mocap dataset.
SOMA is trained to be robust against subject shape and pose variations.
Hence, you can apply the SuperSet model basically on any single subject, body only MPC data that you can put your hands on.

You can either go ahead and mosh the created c3d files, or you grab a c3d frame where you think is
most representative of the markerlayout(s) used for the dataset and train a new SOMA for it.
If you chose to mosh the dataset directly without training a specific SOMA, then we would suggest using
```` mosh_stagei_perseq=True ```` in call for 'run_soma_on_multiple_settings'.
This way MoSh stage-I would be applied on every sequence so in case one label is assigned to different locations MoSh can fix that.
